In [19]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from .config import *

# for in-notebook rendering
%matplotlib inline

# dataset location
root_dir = z_path + "\\fma_medium"


#### notebook functions

In [20]:
def make_file_list(root_dir):
    '''given a directory, loops through folders and files and return list of files paths'''
    file_list = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            file_list.append(os.path.join(subdir, file))

    return file_list

def make_genre_list(song_list):
    '''use tinytag to count number of songs for each genre'''
    genres = {}
    for song in song_list:
        tag = TinyTag.get(song)
        # if tag.genre is not None:
        #     if tag.genre in genres:
        #         genres[tag.genre] += 1
        #     else:
        #         genres[tag.genre] = 1
        
        # what.... why is this a genre
        if tag.genre == 'Popcorn':
            print(song)

    return genres
        

### exploring FMA audio samples

In [ ]:
# short
songs = make_file_list(root_dir)

In [ ]:
# takes time
genres = make_genre_list(songs)

In [ ]:
print(genres)


# change directory to CSV and load data
# data_dir = z_path + "\\fma_metadata\\"
# data_file = "genres.csv"
# os.chdir(data_dir)
# genre_df = pd.read_csv(data_file)

# # find genre roots for music
# genre_heads = genre_df.top_level.unique()